<a href="https://colab.research.google.com/github/ashimasinghal/T5-for-Text-Generation/blob/main/t5_TopK_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==3.5.1

     |████████████████████████████████| 1.3MB 24.5MB/s 
     |████████████████████████████████| 1.1MB 39.6MB/s 
     |████████████████████████████████| 901kB 33.1MB/s 
     |████████████████████████████████| 2.9MB 40.8MB/s 


In [ ]:
!pip install torch==1.4.0

     |████████████████████████████████| 753.4MB 23kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [ ]:

import torch
from transformers import T5ForConditionalGeneration,AutoTokenizer

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def beam_generations(model,input_ids,attention_masks,types):
    if (types=='skipgram'):
        beam_output_skipgram = model.generate(
                            input_ids=input_ids,attention_mask=attention_masks, 
                            max_length=256, 
                            num_beams=5, 
                            no_repeat_ngram_size=2, 
                            early_stopping=True
                                )
        return beam_output_skipgram
    elif(types=="classical"):
        beam_output_classic = model.generate(
                               input_ids=input_ids,attention_mask=attention_masks, 
                                max_length=256, 
                                num_beams=5,
                                early_stopping=True
                                )
        return beam_output_classic
    elif (types=="returnseq"):
        beam_output_returnseq = model.generate(
                                input_ids=input_ids,attention_mask=attention_masks, 
                                num_beams=50, 
                                no_repeat_ngram_size=2,
                                num_return_sequences=10,     
                                early_stopping=True
                                )
        return beam_output_returnseq
    
def sampling_generations(model,input_ids,attention_masks,types):
    if types=="nucleus":
        sample_nucleus = model.generate(
                        input_ids=input_ids,attention_mask=attention_masks,
                        do_sample=True, 
                        max_length=256, 
                        top_p=0.98, 
                        top_k=120
                    )
        return sample_nucleus
    elif types=="kpsampling":
        sample_outputskp = model.generate(
                        input_ids=input_ids,attention_mask=attention_masks,
                        do_sample=True, 
                        max_length=256, 
                        top_k=120, 
                        top_p=0.98, 
                        early_stopping=True,
                        num_return_sequences=10
        )
        return sample_outputskp

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install sentencepiece

In [ ]:
import pandas as pd

file=open("/content/gdrive/MyDrive/search_terms_tweets_cleaned.txt","r")
sentence_li=[]
while True:
    l=file.readline()
    if type(l)==str:
        sentence_li.append(l)
    if not l:
        break
file.close()
sentence_li=[w.strip("\n") for w in sentence_li]       
        
set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = AutoTokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

temporal_list=[]
for i in sentence_li:
    sentence=i
    sub_list=[]

    text =  "paraphrase: " + sentence + " </s>"


    max_len = 256

    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
#     beam_outputs = model.generate(
#     input_ids=input_ids, attention_mask=attention_masks,
#     do_sample=True,
#     max_length=256,
#     top_k=120,
#     top_p=0.98,
#     early_stopping=True,
#     num_return_sequences=10
#     )
    #beam_outputs=beam_generations(model,input_ids,attention_masks,"returnseq")
    beam_outputs=beam_generations(model,input_ids,attention_masks,"classical")
    print(beam_outputs)
    print ("\nOriginal Question ::")
    print (sentence)
    print ("\n")
    print ("Paraphrased Questions :: ")
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)

    for i, final_output in enumerate(final_outputs):
        print("{}: {}".format(i, final_output))
        sub_list.append(final_output)
    temporal_list.append(sub_list)

sample_df=pd.DataFrame(columns=["Actual","Predicted"])
sample_df["Actual"]=sentence_li
sample_df["Predicted"]=temporal_list
sample_df.to_csv("/content/gdrive/MyDrive/paraphrase_t5_topK_sample_output.csv",index=False)
sample_df



device  cuda


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Streaming output truncated to the last 5000 lines.
Paraphrased Questions :: 
0: Singapore to Review Hong Kong Travel Bubble as Virus Returns.
tensor([[    0,  6243,    56,  1132,   165,   799,  1111, 11144,    28,  7764,
          7807,     6,    38,     8,   684,    19,  8191,    12,  6269,     8,
          3060,    13,     3,     9,   126,  6826,   166,  4313,    16,  1547,
             6,    57,     3, 26781,     3,     9,   386,    18,  8041,  5261,
          3035,   379,     3, 17979,   569,  7241,     7,    11,  4095,    35,
            53,  4947, 16110,     7,     5,     1]], device='cuda:0')

Original Question ::
Singapore is to review its air travel bubble with Hong Kong, as the country is racing to stem the spread of a new variant first identified in India, by imposing a three-week crackdown including limiting social gatherings and tightening border curbs.


Paraphrased Questions :: 
0: Singapore will review its air travel bubble with Hong Kong, as the country is racing to st

,Actual,Predicted
0,Sources from Noida media groups are saying Sar...,[Sources from Noida media groups are saying Sa...
1,"Sonu Sood, Team Save Lives Of twenty-two Covid...","[Sonu Sood, Team Save Lives Of Twenty-two Covi..."
2,We are too busy with Covid management to waste...,[We are too busy with Covid management to wast...
3,Madhya Pradesh- Covid Patients getting World C...,[Madhya Pradesh- Covid Patients getting World ...
4,The Department of Telecommmunications has give...,[The Department of Telecommmunications has giv...
...,...,...
6066,"This is the thing, these small businesses are ...",[]
6067,Circuit breaker??....Kenny should not use elec...,[Circuit breaker?....Kenny should not use elec...
6068,"We have seen the enemy, and he is Jason Kenney...","[We have seen the enemy, and he is Jason Kenne..."
6069,"My kid echoed comment when he told my Mom, ""He...","[My kid echoed comment when he told my Mom, ""H..."
